# F-of-M Analysis: Top 50 Census Sections

This notebook performs a Figure of Merit (F-of-M) analysis to identify the 50 census sections with the most suitable demographic profile for senior residences.

## Objectives:
1. Load F-of-M data and geographic coordinates
2. Select the top 50 sections according to F-of-M

## Requirements:
- File: `VARIABLES_SEED.xlsx` (must be in the same directory)
- Libraries: pandas, matplotlib, numpy, openpyxl

---

## Install dependencies

In [1]:
try:
    import pandas as pd
    import folium
    from folium.plugins import MarkerCluster
except ImportError as e:
    print(f"A library is missing: {e}")
    !pip install pandas folium openpyxl -q
    print("Installation completed. Please restart the kernel and run again.")

## Upload data

In [2]:
import pandas as pd
from pathlib import Path

ARCHIVO_DATOS = Path("/home/lgarbayo/Escritorio/mvp-residence/data/VARIABLES SEED.xlsx")

if not Path(ARCHIVO_DATOS).exists():
    raise FileNotFoundError(f"File not found: {ARCHIVO_DATOS}")

df = pd.read_excel(ARCHIVO_DATOS, sheet_name=0)
df = df.rename(columns={
    'id_seccion': 'seccion_censal',
    'length': 'longitud',
    'latitude': 'latitud'
})

## Select top 50 sections

In [3]:
# sort by F-of-M ascending (lower is better)
df_sorted = df.sort_values('f_of_m', ascending=True).reset_index(drop=True)

# select top 50
df_top50 = df_sorted.head(50).copy()
df_top50['ranking'] = range(1, 51)

# extract province from census section code
df_top50['provincia'] = df_top50['seccion_censal'].astype(str).str.zfill(10).str[:2]

# code provinces dictionary of INE
PROVINCIAS = {
    '01': 'Álava', '02': 'Albacete', '03': 'Alicante', '04': 'Almería',
    '05': 'Ávila', '06': 'Badajoz', '07': 'Baleares', '08': 'Barcelona',
    '09': 'Burgos', '10': 'Cáceres', '11': 'Cádiz', '12': 'Castellón',
    '13': 'Ciudad Real', '14': 'Córdoba', '15': 'A Coruña', '16': 'Cuenca',
    '17': 'Girona', '18': 'Granada', '19': 'Guadalajara', '20': 'Gipuzkoa',
    '21': 'Huelva', '22': 'Huesca', '23': 'Jaén', '24': 'León',
    '25': 'Lleida', '26': 'La Rioja', '27': 'Lugo', '28': 'Madrid',
    '29': 'Málaga', '30': 'Murcia', '31': 'Navarra', '32': 'Ourense',
    '33': 'Asturias', '34': 'Palencia', '35': 'Las Palmas', '36': 'Pontevedra',
    '37': 'Salamanca', '38': 'S.C. Tenerife', '39': 'Cantabria', '40': 'Segovia',
    '41': 'Sevilla', '42': 'Soria', '43': 'Tarragona', '44': 'Teruel',
    '45': 'Toledo', '46': 'Valencia', '47': 'Valladolid', '48': 'Bizkaia',
    '49': 'Zamora', '50': 'Zaragoza', '51': 'Ceuta', '52': 'Melilla'
}

df_top50['nombre_provincia'] = df_top50['provincia'].map(PROVINCIAS).fillna('null')

## Export data

In [4]:
# uutput directory
output_dir = Path('outputs')
output_dir.mkdir(exist_ok=True)

df_export = df_top50.copy()
df_export['seccion_censal'] = df_export['seccion_censal'].astype(str).str.zfill(10)

output_csv = output_dir / 'top50_secciones_fofm.csv'
df_export.to_csv(output_csv, index=False, encoding='utf-8')

## Generate map with folium

In [5]:
import folium
from folium.plugins import MarkerCluster

map = folium.Map(
    location=[40.4168, -3.7038],
    zoom_start=6,
    tiles='OpenStreetMap'
)

# add markers for each section
for _, row in df_top50.iterrows():
    rank = int(row['ranking'])
    
    # colour and size
    if rank <= 10:
        color = 'green'
        icon_type = 'star'
        icon_prefix = 'fa'
        emoji = '🥇'
    elif rank <= 25:
        color = 'blue'
        icon_type = 'home'
        icon_prefix = 'fa'
        emoji = '🔵'
    else:
        color = 'orange'
        icon_type = 'home'
        icon_prefix = 'fa'
        emoji = '🟠'
    
    # popup html
    popup_html = f"""
    <div style='font-family: Arial; font-size: 13px; min-width: 250px;'>
        <h4 style='margin:0 0 10px 0; color: #2C3E50;'>{emoji} Ranking #{rank}</h4>
        <hr style='margin: 10px 0;'>
        <table style='width:100%; border-collapse: collapse;'>
            <tr>
                <td style='padding: 5px 0;'><strong>Sección Censal:</strong></td>
                <td style='padding: 5px 0;'>{int(row['seccion_censal'])}</td>
            </tr>
            <tr style='background-color: #f8f9fa;'>
                <td style='padding: 5px 0;'><strong>F-of-M:</strong></td>
                <td style='padding: 5px 0;'>{row['f_of_m']:.4f}</td>
            </tr>
            <tr>
                <td style='padding: 5px 0;'><strong>Provincia:</strong></td>
                <td style='padding: 5px 0;'>{row['nombre_provincia']}</td>
            </tr>
            <tr style='background-color: #f8f9fa;'>
                <td style='padding: 5px 0;'><strong>Coordenadas:</strong></td>
                <td style='padding: 5px 0;'>{row['latitud']:.4f}, {row['longitud']:.4f}</td>
            </tr>
        </table>
    </div>
    """
    
    # create marker
    folium.Marker(
        location=[row['latitud'], row['longitud']],
        popup=folium.Popup(popup_html, max_width=350),
        tooltip=f"{emoji} #{rank}: {row['nombre_provincia']} (F-of-M: {row['f_of_m']:.4f})",
        icon=folium.Icon(color=color, icon=icon_type, prefix=icon_prefix)
    ).add_to(map)

# add layer control
folium.LayerControl().add_to(map)

# save map
output_mapa = output_dir / 'mapa_interactivo_top50.html'
map.save(str(output_mapa))

## View map

In [6]:
map